<a href="https://colab.research.google.com/github/KunaCornejo/SmartGatewaySelection/blob/main/SmartSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy import genfromtxt
from geopy.distance import great_circle

In [2]:
#import git
!git clone https://github.com/KunaCornejo/SmartGatewaySelection.git


Cloning into 'SmartGatewaySelection'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 33 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (33/33), done.


In [3]:
ls 'SmartGatewaySelection'

c_n_ul.csv  Locaciones.csv  Matriz_Train_90.csv.zip  SmartSelection.ipynb


In [4]:
c_n_ul = genfromtxt('/content/SmartGatewaySelection/c_n_ul.csv', delimiter=',') 
c_n_ul=list(c_n_ul)

In [5]:
ubicaciones=['Panama','San Jose','Tegucigalpa','Mexico City','Monterrey','Guadalajara'\
             ,'Tijuana','La Habana','Sto. Domingo','San Salvador','San Juan',\
             'Torreon','La Paz (BC)','Veracruz','Cancun','Queretaro'\
             ,'Tuxtla (Gtz)','Cd. Juarez','San Pedro Sula','Belmopan','Kingston'\
             ,'Merida','Pto. Cabezas','Oaxaca']
ubicaciones=np.transpose([ubicaciones])
C_N_UL=np.array([c_n_ul],dtype=object) 
C_N_UL=np.transpose(C_N_UL)
C_N_UL=np.delete(C_N_UL, 15, 0) #Get rid of Chihuahua, it is not necessary
CN_UP=np.concatenate((ubicaciones,C_N_UL),axis=1) #Concatenate two lists
print(CN_UP)

[['Panama' 32.4401636]
 ['San Jose' 32.4611016]
 ['Tegucigalpa' 32.4450513]
 ['Mexico City' 32.3918091]
 ['Monterrey' 32.313151]
 ['Guadalajara' 32.3588726]
 ['Tijuana' 32.0981607]
 ['La Habana' 32.3380601]
 ['Sto. Domingo' 32.2887155]
 ['San Salvador' 32.4521009]
 ['San Juan' 32.2438352]
 ['Torreon' 32.301041]
 ['La Paz (BC)' 32.2705558]
 ['Veracruz' 32.4018877]
 ['Cancun' 32.3779865]
 ['Queretaro' 32.3736176]
 ['Tuxtla (Gtz)' 32.4294244]
 ['Cd. Juarez' 32.195777]
 ['San Pedro Sula' 32.4354086]
 ['Belmopan' 32.4215586]
 ['Kingston' 32.3572017]
 ['Merida' 32.3851709]
 ['Pto. Cabezas' 32.4314209]
 ['Oaxaca' 32.4205395]]


In [6]:
# Python code to sort the array using second element  
# of sublist Function to sort using sorted() 
def Sort(sub_li): 
    # reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of  
    # sublist lambda has been used 
    return(sorted(sub_li, key = lambda x: x[1], reverse = True)) 
    #X[] es la columna que vamos a ordenar
# Driver Code 
sub_li = CN_UP
sub_li= np.array(Sort(sub_li))  #Just test the sorted list 
print(sub_li)

[['San Jose' 32.4611016]
 ['San Salvador' 32.4521009]
 ['Tegucigalpa' 32.4450513]
 ['Panama' 32.4401636]
 ['San Pedro Sula' 32.4354086]
 ['Pto. Cabezas' 32.4314209]
 ['Tuxtla (Gtz)' 32.4294244]
 ['Belmopan' 32.4215586]
 ['Oaxaca' 32.4205395]
 ['Veracruz' 32.4018877]
 ['Mexico City' 32.3918091]
 ['Merida' 32.3851709]
 ['Cancun' 32.3779865]
 ['Queretaro' 32.3736176]
 ['Guadalajara' 32.3588726]
 ['Kingston' 32.3572017]
 ['La Habana' 32.3380601]
 ['Monterrey' 32.313151]
 ['Torreon' 32.301041]
 ['Sto. Domingo' 32.2887155]
 ['La Paz (BC)' 32.2705558]
 ['San Juan' 32.2438352]
 ['Cd. Juarez' 32.195777]
 ['Tijuana' 32.0981607]]


In [7]:
#Unzip the .csv file
import zipfile #import ZipFile
with zipfile.ZipFile('/content/SmartGatewaySelection/Matriz_Train_90.csv.zip','r') as zip_ref:
    zip_ref.extractall('content')

In [8]:
#It is imported the predicted rain attenuation matrix obtained from Deep Learning Model 
Matrix_Arain = genfromtxt('content/Matriz_Train_90.csv', delimiter=',')
Matrix_Arain = np.delete(Matrix_Arain, 15, 0) #Get rid of Chihuahua, it is not necessary

In [9]:
#The CNIR Matrix is padding by zeros in order to load values later.
CNIR=np.zeros((len(CN_UP[:,0]),len(Matrix_Arain[0,:])),dtype=float)
Y=np.zeros((len(CN_UP[:,0]),len(Matrix_Arain[0,:])),dtype=int) 
prob=np.zeros((len(CN_UP[:,0]),1),dtype=float)  #Array 24x1

In [10]:
CI_co = 36.09  #C/I co-channel at 50 GHz (offset-paraboli-reflctor antenna simulations)
CI_adj = 38.17 #C/I adjacent at 50 Ghz
trh = 14.00    #CNIR Threshold

#CNIR [C/(N+I)] calculations for each feeder link at time-step t+1 
#(due to the predicted rain attenuation)
#Further, it calculates every feeder link availability 

for i in range(len(CN_UP[:,0])):
    CNIR[i,:]=CN_UP[i,1]-Matrix_Arain[i,0:]
    CNIR[i,:]=10*np.log10(1/(1/(np.power(10,CNIR[i,0:]/10))+1/(np.power(10,CI_co/10))\
        +1/(np.power(10,CI_adj/10))))     
    for j in range(len(CNIR[i,0:])):
        if CNIR[i,j]>=trh: 
            Y[i,j]=1
    prob[i,0]=sum(Y[i,0:])/len(Y[i,0:])
Y=np.array(Y,dtype=object)
estado = np.concatenate((ubicaciones,prob,Y),axis=1)

In [11]:
#To obtain geographic coordinates from each location in order to find
#the distance matrix D
sites = genfromtxt('/content/SmartGatewaySelection/Locaciones.csv', delimiter=',') 
sites=np.array(sites,dtype=float)
sites=np.delete(sites, 15, 0) #Get rid of Chihuahua, it is not necessary
#Creamos Matriz de Distancias entre ubicaciones
D=np.zeros((len(sites[0:,0]),len(sites[0:,0])),dtype=float)

for i in range(len(sites[0:,0])):
    for j in range(len(sites[0:,0])):
        D[i,j]=great_circle((sites[i,0],sites[i,1]),(sites[j,0],sites[j,1])).kilometers

D=np.array(D,dtype=object)
D_tag = np.concatenate((ubicaciones,D),axis=1) #Columna de encabezados
ubic = ubicaciones.tolist() #Convierte un array en una lista
ubic.insert(0,[''])
ubic = np.transpose(ubic)
D_tag = np.concatenate((ubic,D_tag),axis=0) #Fila de encabezados

#To save the distance matrix D
#np.savetxt('.../Distance.csv',D_tag,fmt='%s',delimiter=',')
print('done')

done


In [16]:
#La siguiente fase es ordenar las ubicaciones ordenando primero por probabilidades
#y luego ordenamos por distancias > d (km) según el número N de estaciones nominales

N=4         #Número de Estaciones N nominales, Con N=4, d=125.. da 3963 cortes con el CNIR combinado y 2035 con el CNIR uplink normal (ambos con trh=16.55 dB)
d_trh=80   #Umbral de distancia mínimo

nominal_matrix=np.zeros((N,len(estado[0,0:])),dtype=object)
redundant_matrix=np.zeros(((len(estado[0:,0])-N),len(estado[0,0:])),dtype=object)

estado_ordenado = np.array(Sort(estado))
#np.savetxt('/Volumes/Macintosh HD/Users/andrescornejo/Documents/Doctorado/Python/Cap6-Paper/Ordenado.csv',estado_ordenado[:,0:2],fmt='%s',delimiter=',')

nominal_matrix=estado_ordenado[0:N,0:]
redundant_matrix=estado_ordenado[N:,0:]

filas = [' ']
contador=0
while (filas[0] == ' '):
    contador+=1
    for k in range(0,N):
        site_ini=nominal_matrix[k,0]
        i=list(D_tag[0,0:]).index(site_ini)
        print(f'\nAnalizamos el sitio: {site_ini}\n')
        for l in range(0,N):
            site_comp=nominal_matrix[l,0]
                    
            j=list(D_tag[0:,0]).index(site_comp)
    
            if ((k==l) & (D_tag[i,j]==0)):
                print(f'No se analiza el mismo sitio: {site_comp}')
            elif ((k!=l) & (D_tag[i,j]>=d_trh)):
                print(f'Cumple la condición de distancia hacia {site_comp}: {D_tag[i,j]}')
            elif ((k!=l) & (D_tag[i,j]<d_trh)):
                print(f'No Cumple la condición de distancia hacia {site_comp}: {D_tag[i,j]}')
                filas.append(l)
        
    if len(filas)>1:
        print('\nIntercambio de Filas')
        filas=list(dict.fromkeys(filas)) #Elimina elementos repetidos dentro del array
        
        redundant_matrix_res=redundant_matrix #Respaldamos Nominal Matrix
        
        for n in range(0,len(filas)-1):
            #Enviamos al final de la matriz redundante
            redundant_matrix=np.insert(redundant_matrix,len(estado)-N,nominal_matrix[filas[n+1],0:],axis=0)
            #Intercambiamos el no valida de la matriz nominal con el mejor redundante en probabilidad
            nominal_matrix[filas[n+1],0:] = redundant_matrix_res[n,0:]
            #borramos la fila del apoyo redundante 
            redundant_matrix=np.delete(redundant_matrix,0, axis=0)
        filas.clear()
        filas = [' ']
    else:
        print(f'\nTodos los sitios Cumplen con la Distancia, D>{d_trh} km\n')
        filas.clear()
        filas.insert(0,'done')
        
nominal_matrix=np.array(Sort(nominal_matrix))
redundant_matrix=np.array(Sort(redundant_matrix))

print(nominal_matrix)
print(redundant_matrix)



Analizamos el sitio: Torreon

No se analiza el mismo sitio: Torreon
Cumple la condición de distancia hacia Cd. Juarez: 754.8701336424568

Analizamos el sitio: Cd. Juarez

Cumple la condición de distancia hacia Torreon: 754.8701336424568
No se analiza el mismo sitio: Cd. Juarez

Todos los sitios Cumplen con la Distancia, D>80 km

[['Torreon' 0.9994425418569254 1 ... 1 1 1]
 ['Cd. Juarez' 0.9992161339421614 1 ... 1 1 1]]
[['La Paz (BC)' 0.9991552511415525 1 ... 1 1 1]
 ['Tijuana' 0.9989554794520548 1 ... 1 1 1]
 ['Oaxaca' 0.9986111111111111 1 ... 1 1 1]
 ...
 ['San Juan' 0.9902226027397261 1 ... 1 1 1]
 ['Panama' 0.9898439878234399 1 ... 1 1 1]
 ['Pto. Cabezas' 0.9888831811263318 1 ... 1 1 1]]
